<a href="https://colab.research.google.com/github/Freedomwait/css-static-demo/blob/main/v1/BZZschoolaugdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

# 原始bzz数据-> db2字段dataframe

In [109]:
beijingschools = pd.read_excel('bzz_beijing_university.xlsx')
sichuanschools = pd.read_excel('bzz_sichuan.xlsx').iloc[1:,]
hubeischools = pd.read_excel('bzz_hubei_beijing.xlsx')

In [110]:
lsdf = [beijingschools,sichuanschools,hubeischools]
for i in range(len(lsdf)):
    lsdf[i].columns = lsdf[i].iloc[0]
    lsdf[i] = lsdf[i].iloc[1:,:]
    lsdf[i] = lsdf[i].drop_duplicates()

In [111]:
dfaugschooldata = pd.concat(lsdf, ignore_index=True).reset_index()

In [112]:
colsselected = ['标题','省份','城市', '区县', '发布时间', '项目编号', '招采单位','招采联系人/号码(公告)', '招采联系号码(企查查)','中标单位', '中标金额', '中标候选单位', '中标联系人/号码(公告)','中标联系号码(企查查)', '原文链接']

In [113]:
df = dfaugschooldata.copy()
df = df[colsselected]
df.columns = ['project_name','province_b','county_b','district_b','winning_date','project_id','buyer_name','buyer_contact/phone','buyer_phone(qichacha)','supplier_name','winning_amt','candidates','supplier_contact/phone','supplier_phone(qichacha)','winning_notice_url']

# Explode to Order Level

In [114]:
df['needtoexplode'] = df['supplier_name'].fillna("").apply(lambda x: 1 if '\n' in x else 0)

In [115]:
df['index'] = range(df.shape[0])

In [116]:
dfexplode = df[df['needtoexplode'] == 1][['index','supplier_name','winning_amt']]
dfntexplode1 = df[df['needtoexplode'] != 1][['index','supplier_name','winning_amt']]

In [117]:
groupby_cols = df.columns.tolist()
# remove project_order level columns
order_lvl_cols = ['supplier_name', 'winning_amt']
for c in order_lvl_cols:
    groupby_cols.remove(c)

In [118]:
dfexplode1 = dfexplode.set_index('index').apply(lambda x: x.str.split(',\n').explode()).reset_index()
dforderlvl = pd.concat([dfexplode1,dfntexplode1])
df = dforderlvl.merge(df[groupby_cols],on='index',how='left')[['project_name','province_b','county_b','district_b','winning_date','project_id','buyer_name','buyer_contact/phone','buyer_phone(qichacha)','supplier_name','winning_amt','candidates','supplier_contact/phone','supplier_phone(qichacha)','winning_notice_url']]

# Clean Order Level BZZ Data

In [119]:
df['buyer_contact/phone'] = df['buyer_contact/phone'].fillna('/')
df['buyer_contact'] = df['buyer_contact/phone'].apply(lambda x: x.split('/')[0])
df['buyer_phone'] = df['buyer_contact/phone'].apply(lambda x: x.split('/')[1])

In [120]:
df['supplier_contact/phone'] = df['supplier_contact/phone'].fillna('/')
df['supplier_contact'] = df['supplier_contact/phone'].apply(lambda x: x.split('/')[0])
df['supplier_phone'] = df['supplier_contact/phone'].apply(lambda x: x.split('/')[1])
df['supplier_contact'] = np.where((df['supplier_contact'] == '未公示')|(df['supplier_contact'] == ''), 
                                            np.nan, 
                                            df['supplier_contact'])

In [121]:
col_inddex_loc = df.columns.tolist().index('winning_amt')

In [122]:
# clean winning amt
df['winning_amt'] = df['winning_amt'].fillna('')
index_loc = df.columns.tolist().index('winning_amt')

for i in range(df.shape[0]):
        cur_win_amt = df.iloc[i, index_loc]

        if cur_win_amt == '未公示' or cur_win_amt == '':
                df.iloc[i, index_loc] = np.nan
        elif '万' in cur_win_amt:
                df.iloc[i, index_loc] = float(re.sub('[^\d\.]', "", cur_win_amt))*10000
        else:
                if re.sub('[^\d\.]', "", cur_win_amt) != '':
                        df.iloc[i, index_loc] = float()
                else:
                        df.iloc[i, index_loc] = np.nan

In [123]:
df['winning_amt'] = df['winning_amt'].astype(float)

In [124]:
df.rename({'buyer_contact/phone': 'buyer_contact_phone', 
                    'buyer_phone(qichacha)': 'buyer_phone_qcc', 
                    'supplier_phone(qichacha)': 'supplier_phone_qcc', 
                    'supplier_contact/phone': 'supplier_contact_phone'}, 
                    axis = 'columns', inplace = True)

In [125]:
!pip install duckdb
import duckdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [126]:
dfaugfinaldata = duckdb.query("SELECT project_id \
, project_name \
, province_b as province \
, county_b as county \
, district_b as district \
, winning_date \
, winning_notice_url \
, buyer_name \
, buyer_contact \
, coalesce(buyer_phone, buyer_phone_qcc) as buyer_phone \
, winning_amt \
, supplier_name \
, supplier_contact \
, coalesce(supplier_phone, supplier_phone_qcc) as supplier_phone from df").to_df()

In [127]:
dfaugfinaldata

,project_id,project_name,province,county,district,winning_date,winning_notice_url,buyer_name,buyer_contact,buyer_phone,winning_amt,supplier_name,supplier_contact,supplier_phone
0,OITC-G220311197,北京大学第一教学楼智慧教室改造招标采购项目(第2、第3、第4包）中标公告,北京市,北京市,海淀区,2022-07-19,http://www.ccgp.gov.cn/cggg/zygg/zbgg/202207/t...,北京大学,吴老师,010-62758587,3571680.00,北京竹远科创科技股份有限公司,NaN,
1,OITC-G220311197,北京大学第一教学楼智慧教室改造招标采购项目(第2、第3、第4包）中标公告,北京市,北京市,海淀区,2022-07-19,http://www.ccgp.gov.cn/cggg/zygg/zbgg/202207/t...,北京大学,吴老师,010-62758587,1862991.00,北京市博汇科技股份有限公司,NaN,
2,OITC-G220311197,北京大学第一教学楼智慧教室改造招标采购项目(第2、第3、第4包）中标公告,北京市,北京市,海淀区,2022-07-19,http://www.ccgp.gov.cn/cggg/zygg/zbgg/202207/t...,北京大学,吴老师,010-62758587,1824667.00,北京伟森盛业家具有限公司,NaN,
3,BMCC-ZC22-0711,北京大学智慧校园基础设施建设项目中标公告,北京市,北京市,海淀区,2022-12-18,http://www.ccgp.gov.cn/cggg/zygg/zbgg/202212/t...,北京大学,吴老师,010-62758587,10080000.00,中泰博瑞（北京）机电工程有限公司,NaN,
4,BMCC-ZC22-0711,北京大学智慧校园基础设施建设项目中标公告,北京市,北京市,海淀区,2022-12-18,http://www.ccgp.gov.cn/cggg/zygg/zbgg/202212/t...,北京大学,吴老师,010-62758587,5248576.00,北京神州新桥科技有限公司,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,2207-HXTC-IN1213,北京交通大学环境实验室安全和平台基础建设中标公告,北京市,北京市,海淀区,2022-09-01,http://www.ccgp.gov.cn/cggg/zygg/zbgg/202209/t...,北京交通大学,侯老师,010-51683701,753900.00,北京华煜八方科技有限公司,NaN,
299,BMCC-ZC22-0568,北京交通大学雄安校区智慧校园专项规划咨询服务项目中标公告,北京市,北京市,海淀区,2022-12-07,http://www.ccgp.gov.cn/cggg/dfgg/zbgg/202212/t...,北京交通大学,皇甫老师,010-51684375,1729800.00,中国建筑科学研究院有限公司,NaN,
300,JSJL2022-050,北京交通大学（威海）智慧校园建设项目中标公示,山东省,威海市,文登区,2022-08-23,https://bulletin.cebpubservice.com/resultBulle...,北京交通大学,谢老师,18063106657,9378576.00,青岛天高智慧科技有限公司,NaN,
301,NaN,北京工业大学校本部西区给水管网监测系统改造中标公告,北京市,北京市,朝阳区,2022-04-07,http://www.ccgp.gov.cn/cggg/dfgg/zbgg/202204/t...,北京工业大学,,,1066407.46,禹王（北京）科技工程有限公司,NaN,


In [128]:
dfaugfinaldata = dfaugfinaldata.drop_duplicates(subset=['project_name','supplier_name'])

In [129]:
dfaugfinaldata['govurl'] = dfaugfinaldata['winning_notice_url'].apply(lambda x: 1 if 'gov.cn' in x else 0)

<ipython-input-129-1a956e2cf89b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfaugfinaldata['govurl'] = dfaugfinaldata['winning_notice_url'].apply(lambda x: 1 if 'gov.cn' in x else 0)


In [130]:
dffinal = dfaugfinaldata[~dfaugfinaldata['supplier_name'].isnull()]

In [131]:
dffinal.columns

Index(['project_id', 'project_name', 'province', 'county', 'district',
       'winning_date', 'winning_notice_url', 'buyer_name', 'buyer_contact',
       'buyer_phone', 'winning_amt', 'supplier_name', 'supplier_contact',
       'supplier_phone', 'govurl'],
      dtype='object')

# 添加学校类型

In [132]:
uni_list = pd.read_csv("university_all_cleaned.csv")
voc_list = pd.read_csv('vocational_inscope - Sheet1.csv')
inscope_schools = uni_list['school_name'].tolist() + voc_list['school_name'].tolist()
insscope_schools_df = pd.concat([voc_list[['school_name', 'school_level']], 
                                 uni_list[['school_name', 'school_level', 'sec_lvl']]   
                                ], 
                                axis = 0)

In [133]:
dffinal = dffinal.merge(insscope_schools_df, how = 'left', left_on = 'buyer_name', right_on = 'school_name')
dffinal['school_level'] = dffinal['school_level'].fillna('other')
dffinal['sec_lvl'] = dffinal['sec_lvl'].fillna(0)
dffinal['sec_lvl'] = dffinal['sec_lvl'].astype(int)

# 添加采购的行政区域

In [134]:
supplierandbuyer1 = pd.read_csv('supplierbuyerinfo.csv')
supplierandbuyer2 = pd.read_csv('subpplierbuyerinfo2.csv')
supplierandbuyer3 = pd.read_csv('supplierbuyerinfo3.csv')
supplierandbuyer4 = pd.read_csv("supplierbuyerinfo4.csv")
supplierandbuyer = pd.concat([supplierandbuyer1,supplierandbuyer2,supplierandbuyer3,supplierandbuyer4])

In [135]:
sbcols = ['企业名称', '所属省份', '所属城市', '所属区县','法定代表人',
'电话', '企业地址']
dfsbinfo = supplierandbuyer[sbcols].rename(columns={'企业名称':'name', '登记状态':'status','所属省份':'province', '所属城市':'county', '所属区县':'district',
'法定代表人':'contact','电话':'phone', '企业地址':'addr'})

In [136]:
dffinal = dffinal.merge(dfsbinfo,left_on='buyer_name',right_on='name',how = 'left',suffixes=['','_b'])\
        .merge(dfsbinfo,left_on='supplier_name',right_on = 'name',how = 'left',suffixes=['','_s'])

In [137]:
dffinal.columns

Index(['project_id', 'project_name', 'province', 'county', 'district',
       'winning_date', 'winning_notice_url', 'buyer_name', 'buyer_contact',
       'buyer_phone', 'winning_amt', 'supplier_name', 'supplier_contact',
       'supplier_phone', 'govurl', 'school_name', 'school_level', 'sec_lvl',
       'name', 'province_b', 'county_b', 'district_b', 'contact', 'phone',
       'addr', 'name_s', 'province_s', 'county_s', 'district_s', 'contact_s',
       'phone_s', 'addr_s'],
      dtype='object')

In [138]:
dffinal['buyer_addr'] = dffinal['addr']
dffinal['buyer_contact'] = dffinal['buyer_contact'].fillna(dffinal['contact'])
dffinal['buyer_phone'] = dffinal['buyer_phone'].fillna(dffinal['phone'])
dffinal['supplier_addr'] = dffinal['addr_s']
dffinal['supplier_phone'] = dffinal['phone_s']
dffinal['supplier_contact'] = dffinal['contact_s']

In [139]:
dffinal['candidates'] = dffinal['supplier_name']

In [140]:
finalcols = ['project_id', 'order_name', 'winning_date', 'winning_amt', 'buyer_name',
       'supplier_name','supplier_addr','supplier_contact','supplier_phone','winning_notice_url', 'buyer_addr', 'buyer_contact',
       'buyer_phone', 'province', 'county', 'district', 'province_s',
       'county_s', 'district_s','candidates','school_level','sec_lvl']

In [141]:
schoolaug = dffinal.rename(columns = {'project_name':'order_name'})[finalcols]

In [142]:
schoolaug.to_csv("schoolaugdata.csv",index=False)